## Linear Growth Approximation

#### Approximation only suitable for steady-state (SS)

The _linear growth approximation_ is an approach which assumes that over some time step X, the growth is approximately linear. This means that the removal constant must be calculated such that the number of atoms at time X is the same as in the SaltProc results at that time (post-removal). As long as accuracy can be maintained to some position past where the SaltProc value is determined, then this method can be used to reduce the computational time of SaltProc by making a series of small linear approximations.

![title](LGA.png)

This approach has 3 main steps

1. Run Serpent with no reprocessing for X days

    1a. This gives enough data to create an approximate linear growth term
    
    $$\frac{dN}{dt} = C$$

    $$C = \frac{N_f - N_i}{t_f - t_i}$$

2. Plug the constant growth value into the Bateman equation and then include reprocessing with the condition that it must match the SaltProc result at a given time. This is done by calculating waste atoms and solving with the `scipy.optimize` root function.

    $$\frac{dN}{dt} = C - \lambda N$$

    $$N(t) = N_i e^{-\lambda t} + \frac{C}{\lambda} \left( 1 - e^{-\lambda t}  \right)$$
    
    $$\frac{dN_{waste}}{dt} = \lambda N(t)$$

    $$\frac{dN_{waste}}{dt} = \lambda (N_i e^{-\lambda t} + \frac{C}{\lambda} \left( 1 - e^{-\lambda t}  \right))$$

    $$N_{waste}(t) = C_1 + Ct + \frac{C e^{-\lambda t}}{\lambda} - N_0 e^{-\lambda t}$$
    
    Using:

    $$C = \frac{N_f - N_i}{t_f - t_i}$$
    
    $$C_1 = N_0 - \frac{C}{\lambda}$$
    
    $$N_{waste}(t) = N_{linear}(t) - N_{SaltProc}(t)$$
    

3. Run Serpent using type 1 flows and the calculated reprocessing constants

## Cycle Time Decay Approximation

#### Suitable for any time scale

This approximation treats the type 1 flow as a decay (which is how it is viewed mathematically). This means that the "decay" constant can be calculated from the "half-life" of a particular target.

In this case, the half-lives are determined from the cycle times from Robertson et al. report on the MSBR that Andrei cites in the MSBR example (R. C. Robertson. Conceptual Design Study of a Single-Fluid Molten-Salt Breeder
Reactor. Technical Report ORNL-4541, ORNL, January 1971.)

![title](cycle_time.PNG)

| Element | Cycle Time (s) | "Half-life" | "Decay" Constant |
| :- | :-: | :-: | :-: | 
| Kr | 20 | 10 | 6.93147E-2 |
| Xe | 20 | 10 | 6.93147E-2 |
| Se | 20 | 10 | 6.93147E-2 |
| Nb | 20 | 10 | 6.93147E-2 |
| Mo | 20 | 10 | 6.93147E-2 |
| Tc | 20 | 10 | 6.93147E-2 |
| Ru | 20 | 10 | 6.93147E-2 |
| Rh | 20 | 10 | 6.93147E-2 |
| Pd | 20 | 10 | 6.93147E-2 |
| Ag | 20 | 10 | 6.93147E-2 |
| Sb | 20 | 10 | 6.93147E-2 |
| Te | 20 | 10 | 6.93147E-2 |
| Cd | 17280000 | 8640000 | 8.02254E-8 |
| In | 17280000 | 8640000 | 8.02254E-8 |
| Sn | 17280000 | 8640000 | 8.02254E-8 |
| Br | 5184000 | 2592000 | 2.67418E-7 |
| I | 5184000 | 2592000 | 2.67418E-7 |
| Pa | 259200 | 129600 | 5.34836E-6 |
| Y | 4320000 | 2160000 | 3.20901E-7 |
| La | 4320000 | 2160000 | 3.20901E-7 |
| Ce | 4320000 | 2160000 | 3.20901E-7 |
| Pr | 4320000 | 2160000 | 3.20901E-7 |
| Nd | 4320000 | 2160000 | 3.20901E-7 |
| Pm | 4320000 | 2160000 | 3.20901E-7 |
| Sm | 4320000 | 2160000 | 3.20901E-7 |
| Gd | 4320000 | 2160000 | 3.20901E-7 |
| Eu | 4320000 | 2160000 | 3.20901E-7 |
| Rb | 296784000 | 148392000 | 4.67105E-9 |
| Sr | 296784000 | 148392000 | 4.67105E-9 |
| Cs | 296784000 | 148392000 | 4.67105E-9 |
| Ba | 296784000 | 148392000 | 4.67105E-9 |


These "decay" constants are then plugged in as the reprocessing constants


## Isotope-Based Linear Growth Approximation

#### Only suitable for SS

This approximation selects the "most important" isotopes from each element removal and sets the removal rate based on the Bateman equation for that isotope.

Since some elements may not have isotopes which are necessarily important enough to track that closely (for neutronics purposes), we will set any element without important isotopes to use the base Linear Growth Approximation.

[Some cross section data to use for determining other important isotopes]( https://www.osti.gov/etdeweb/servlets/purl/20332542 )

Xenon 135, Samrium 149 are two significant isotopes.
Xe-135 comes from I-135 decay.

These three will be the main focus for now, further isotopes/elements can be added later.

| Element | Isotope |
| : -     |  : - :  |
| Xe | 135 |
| Sm | 149 |
| I  | 135 |

The process is the same as the Linear Growth Approximation, but it calculates each element's reprocessing constant based on the results of the important isotopes.



## Efficiency-Based Removal

Using the cycle time data, a linear approximation is used to determine what removal percentage per second would result in the cycle times given.

![title](cycle_time.PNG)

To calculate the removal efficiency, the following approach is used:

    Cycle time of 1s -> remove 100% per second
    Cycle time of 2s -> remove 50% per second
    Cycle time of 4s -> remove 25% per second
    ...
    Cycle time of Ts -> remove 1/T% per second

To calculate the reprocessing constant to use in Serpent, the following approach is used:

$$\lambda_{repr} = ln \left(\frac{1}{1-X} \right)$$

    Where X is the removal efficiency




| Element | Cycle Time (s) | Removal Efficiency (1/s) | Reprocessing Constant |
| :- | :-: | :-: | :-: | 
| Kr | 20 | 0.05 | 5.12933E-2 |
| Xe | 20 | 0.05 | 5.12933E-2 |
| Se | 20 | 0.05 | 5.12933E-2 |
| Nb | 20 | 0.05 | 5.12933E-2 |
| Mo | 20 | 0.05 | 5.12933E-2 |
| Tc | 20 | 0.05 | 5.12933E-2 |
| Ru | 20 | 0.05 | 5.12933E-2 |
| Rh | 20 | 0.05 | 5.12933E-2 |
| Pd | 20 | 0.05 | 5.12933E-2 |
| Ag | 20 | 0.05 | 5.12933E-2 |
| Sb | 20 | 0.05 | 5.12933E-2 |
| Te | 20 | 0.05 | 5.12933E-2 |
| Cd | 17280000 | 5.78704E-8 | 5.78704E-8 |
| In | 17280000 | 5.78704E-8 | 5.78704E-8 |
| Sn | 17280000 | 5.78704E-8 | 5.78704E-8 |
| Br | 5184000 | 1.92901E-7 | 1.92901E-7 |
| I | 5184000 | 1.92901E-7 | 1.92901E-7 |
| Pa | 259200 | 3.85802E-6 | 3.85802E-6 |
| Y | 4320000 | 2.31481E-7 | 2.31481E-7 |
| La | 4320000 | 2.31481E-7 | 2.31481E-7 |
| Ce | 4320000 | 2.31481E-7 | 2.31481E-7 |
| Pr | 4320000 | 2.31481E-7 | 2.31481E-7 |
| Nd | 4320000 | 2.31481E-7 | 2.31481E-7 |
| Pm | 4320000 | 2.31481E-7 | 2.31481E-7 |
| Sm | 4320000 | 2.31481E-7 | 2.31481E-7 |
| Gd | 4320000 | 2.31481E-7 | 2.31481E-7 |
| Eu | 4320000 | 2.31481E-7 | 2.31481E-7 |
| Rb | 296784000 | 3.36945E-9 | 3.36945E-9 |
| Sr | 296784000 | 3.36945E-9 | 3.36945E-9 |
| Cs | 296784000 | 3.36945E-9 | 3.36945E-9 |
| Ba | 296784000 | 3.36945E-9 | 3.36945E-9 |

## SaltProc Efficiency Matching

Similar to the previous approach, but instead uses the efficiencies that SaltProc also uses.

Linearly approximate, i.e.

    100% per 3 days -> 33.333% per day -> 3.85802E-4 1/s

This results in removal efficiencies that vary significantly from the previous model (i.e. Kr) or are nearly the same (i.e. I)

As shown previously as well, the reprocessing constants will be approximately the removal efficiency values since they are sufficiently small.

| Element | SaltProc Batch Efficiency (1/3 days) | Removal Efficiency (1/s) |
| :- | :-: | :-: | 
| Kr | 0.911522 | 3.51667E-6|
| Xe | 0.914857 | 3.52954E-6 |
| Se | 1 | 3.85825E-6 |
| Nb | 1 | 3.85825E-6 |
| Mo | 1 | 3.85825E-6 |
| Tc | 1 | 3.85825E-6 |
| Ru | 1 | 3.85825E-6 |
| Rh | 1 | 3.85825E-6 |
| Pd | 1 | 3.85825E-6 |
| Ag | 1 | 3.85825E-6 |
| Sb | 1 | 3.85825E-6 |
| Te | 1 | 3.85825E-6 |
| Cd | 0.015 | 5.78704E-8 |
| In | 0.015 | 5.78704E-8 |
| Sn | 0.015 | 5.78704E-8 |
| Br | 0.05 | 1.92901E-7 |
| I | 0.05 | 1.92901E-7 |
| Pa | 0.95 | 3.66512E-6 |
| Y | 0.6 | 2.31481E-6 |
| La | 0.6 | 2.31481E-6 |
| Ce | 0.6 | 2.31481E-6 |
| Pr | 0.6 | 2.31481E-6 |
| Nd | 0.6 | 2.31481E-6 |
| Pm | 0.6 | 2.31481E-6 |
| Sm | 0.6 | 2.31481E-6 |
| Gd | 0.6 | 2.31481E-6 |
| Eu | 0.06 | 2.31481E-7 |
| Rb | 0.09 | 3.47222E-7 |
| Sr | 0.09 | 3.47222E-7 |
| Cs | 0.09 | 3.47222E-7 |
| Ba | 0.09 | 3.47222E-7 |